In [1]:
from __future__ import print_function
from openpathsampling.experimental.storage import Storage, ops_class_info, ops_schema
from openpathsampling.experimental.simstore import SQLStorageBackend
import numpy as np
from openpathsampling.experimental.simstore.serialization_helpers import get_uuid, set_uuid

import openpathsampling as paths

import collections

from openpathsampling.experimental.simstore.storable_functions import (
    StorableFunction, StorageFunctionHandler
)

In [2]:
import logging
import sys

root = logging.getLogger()
root.setLevel(logging.DEBUG)

ch = logging.StreamHandler(sys.stdout)
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
root.addHandler(ch)

## Playing around with a storable function

In [3]:
def get_x(snap):
    from time import sleep
    sleep(2)
    return snap.xyz[0][0]

In [4]:
cv = StorableFunction(get_x)

In [5]:
import openpathsampling.engines.toy as toys
snap = toys.Snapshot(coordinates=np.array([[3.0, 0.0]]))

In [6]:
%%time
cv(snap)

2020-11-09 14:12:37,083 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7fa4e8fcaac8>>
2020-11-09 14:12:37,085 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_storage of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7fa4e8fcaac8>>
2020-11-09 14:12:37,088 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._eval of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7fa4e8fcaac8>>
CPU times: user 6.21 ms, sys: 2.81 ms, total: 9.02 ms
Wall time: 2.01 s


3.0

In [7]:
%%time
cv(snap)

2020-11-09 14:12:39,120 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7fa4e8fcaac8>>
CPU times: user 1.18 ms, sys: 906 µs, total: 2.08 ms
Wall time: 1.51 ms


3.0

## Manually registering storable function with SQL backend

In [8]:
backend = SQLStorageBackend("test.sql", mode='w', echo=False)
storage = Storage.from_backend(
    backend=backend,
    schema=ops_schema,
    class_info=ops_class_info,
)

2020-11-09 14:12:39,157 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table uuid
2020-11-09 14:12:39,158 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table tables
2020-11-09 14:12:39,169 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table samples
2020-11-09 14:12:39,178 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table sample_sets
2020-11-09 14:12:39,183 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table trajectories
2020-11-09 14:12:39,190 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table move_changes
2020-11-09 14:12:39,201 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table steps
2020-11-09 14:12:39,208 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table details
2020-11-09 14:12:39,215 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table s

In [9]:
uuid = get_uuid(cv)

In [10]:
_ = cv(snap)
cv.local_cache.result_dict

2020-11-09 14:12:39,307 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7fa4e8fcaac8>>


{'80100521955429081778427985284781047824': 3.0}

In [11]:
backend.register_storable_function(table_name=uuid, result_type='float')

2020-11-09 14:12:39,320 - openpathsampling.experimental.simstore.sql_backend - INFO - Registering storable function: UUID: 80100521955429081778427985284781047820 (float)


In [12]:
backend.add_storable_function_results(table_name=uuid,
                                      result_dict=cv.local_cache.result_dict)

2020-11-09 14:12:39,412 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 0 UUIDs


In [13]:
backend.load_storable_function_results(uuid, [get_uuid(snap)])

2020-11-09 14:12:39,433 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 1 UUIDs


{'80100521955429081778427985284781047824': 3.0}

In [14]:
backend.load_storable_function_table(get_uuid(cv))

{'80100521955429081778427985284781047824': 3.0}

## New backend; use storable func with storage

This will be roughly the full integration.

In [15]:
backend = SQLStorageBackend("test.sql", mode='w')
storage = Storage.from_backend(
    backend=backend,
    schema=ops_schema,
    class_info=ops_class_info
)

2020-11-09 14:12:39,467 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table uuid
2020-11-09 14:12:39,469 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table tables
2020-11-09 14:12:39,490 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table samples
2020-11-09 14:12:39,504 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table sample_sets
2020-11-09 14:12:39,534 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table trajectories
2020-11-09 14:12:39,571 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table move_changes
2020-11-09 14:12:39,585 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table steps
2020-11-09 14:12:39,596 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table details
2020-11-09 14:12:39,607 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table s

In [16]:
backend.identifier

('sqlite:///test.sql', 'w')

In [17]:
storage.save(cv)

2020-11-09 14:12:39,774 - openpathsampling.experimental.simstore.storage - DEBUG - Starting save
2020-11-09 14:12:39,781 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 1 UUIDs
2020-11-09 14:12:39,783 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 1 UUIDs
2020-11-09 14:12:39,790 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 0 UUIDs
2020-11-09 14:12:39,792 - openpathsampling.experimental.simstore.storage - DEBUG - Listing all objects to save
2020-11-09 14:12:39,801 - openpathsampling.experimental.simstore.storage - DEBUG - Checking if objects already exist in database
2020-11-09 14:12:39,803 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 2 UUIDs
2020-11-09 14:12:39,809 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 2 UUIDs
2020-11-09 14:12:39,816 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 0 UUIDs
2020-11-09 14:12:39,819 - openp

In [18]:
list(storage.simulation_objects)

[]

In [19]:
backend.metadata.tables.keys()

dict_keys(['schema', 'metadata', 'uuid', 'tables', 'samples', 'sample_sets', 'trajectories', 'move_changes', 'steps', 'details', 'storable_functions', 'simulation_objects', 'storage_objects', '80100521955429081778427985284781047820'])

In [20]:
storage._sf_handler.functions[0].local_cache.result_dict

{'80100521955429081778427985284781047824': 3.0}

In [21]:
backend.load_storable_function_results(uuid, [get_uuid(snap)])

2020-11-09 14:12:39,953 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 1 UUIDs


{'80100521955429081778427985284781047824': 3.0}

In [22]:
storage.close()

In [23]:
backend = SQLStorageBackend("test.sql", mode='r')
storage = Storage.from_backend(
    backend=backend,
    schema=ops_schema,
    class_info=ops_class_info
)

2020-11-09 14:12:40,172 - openpathsampling.experimental.simstore.storage - INFO - Missing info from 0 dynamically-registered tables
2020-11-09 14:12:40,174 - openpathsampling.experimental.storage.ops_storage - INFO - Found 0 possible lookups
2020-11-09 14:12:40,176 - openpathsampling.experimental.storage.ops_storage - INFO - Lookups for tables: dict_keys([])
2020-11-09 14:12:40,178 - openpathsampling.experimental.simstore.storage - INFO - Successfully registered 0 missing tables
2020-11-09 14:12:40,192 - openpathsampling.experimental.simstore.storage - DEBUG - Starting to load 0 objects
2020-11-09 14:12:40,193 - openpathsampling.experimental.simstore.storage - DEBUG - Getting internal structure of 0 non-cached objects
2020-11-09 14:12:40,199 - openpathsampling.experimental.simstore.storage - DEBUG - Loading 0 objects; creating 0 lazy proxies
2020-11-09 14:12:40,201 - openpathsampling.experimental.simstore.storage - DEBUG - Identifying classes for 0 lazy proxies
2020-11-09 14:12:40,204 

In [24]:
cv_reloaded = storage.load([get_uuid(cv)])[0]

2020-11-09 14:12:40,236 - openpathsampling.experimental.simstore.storage - DEBUG - Starting to load 1 objects
2020-11-09 14:12:40,239 - openpathsampling.experimental.simstore.storage - DEBUG - Getting internal structure of 1 non-cached objects
2020-11-09 14:12:40,241 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 1 UUIDs
2020-11-09 14:12:40,246 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 1 UUIDs
2020-11-09 14:12:40,253 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 1 UUIDs
2020-11-09 14:12:40,257 - openpathsampling.experimental.simstore.storage - DEBUG - Loading 1 objects; creating 0 lazy proxies
2020-11-09 14:12:40,260 - openpathsampling.experimental.simstore.storage - DEBUG - Identifying classes for 0 lazy proxies
2020-11-09 14:12:40,262 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 0 UUIDs
2020-11-09 14:12:40,265 - openpathsampling.experimental.simstore.sql_backend - DEBUG 

In [25]:
cv_reloaded.local_cache.result_dict

{}

In [26]:
cv_reloaded.local_cache != cv.local_cache

True

In [27]:
%%time
cv_reloaded.local_cache.clear()
cv_reloaded(snap)

2020-11-09 14:12:40,331 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7fa4e91255c0>>
2020-11-09 14:12:40,334 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_storage of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7fa4e91255c0>>
2020-11-09 14:12:40,341 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 1 UUIDs
CPU times: user 6.26 ms, sys: 2.57 ms, total: 8.84 ms
Wall time: 11.2 ms


3.0

In [28]:
# TODO: tests need a test of a bad load as well (missing should show up)

In [29]:
cv_reloaded.local_cache.clear()
cv_reloaded.preload_cache()

In [30]:
%%time
cv_reloaded(snap)

2020-11-09 14:12:40,393 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7fa4e91255c0>>
CPU times: user 1.34 ms, sys: 1.33 ms, total: 2.68 ms
Wall time: 2.32 ms


3.0

## Store/load multiple values at once

This checks that the backend is able to load multiple UUIDs at once

In [31]:
backend = SQLStorageBackend("test.sql", mode='w')
storage = Storage.from_backend(
    backend=backend,
    schema=ops_schema,
    class_info=ops_class_info
)

2020-11-09 14:12:40,430 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table uuid
2020-11-09 14:12:40,433 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table tables
2020-11-09 14:12:40,445 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table samples
2020-11-09 14:12:40,454 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table sample_sets
2020-11-09 14:12:40,464 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table trajectories
2020-11-09 14:12:40,476 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table move_changes
2020-11-09 14:12:40,487 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table steps
2020-11-09 14:12:40,495 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table details
2020-11-09 14:12:40,507 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table s

In [32]:
new_snap = toys.Snapshot(coordinates=np.array([[2.0, 0.0]]))

In [33]:
%%time
cv.local_cache.clear()
cv._handler = None
cv([snap, new_snap])

2020-11-09 14:12:40,649 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7fa4e8fcaac8>>
2020-11-09 14:12:40,656 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_storage of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7fa4e8fcaac8>>
2020-11-09 14:12:40,659 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._eval of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7fa4e8fcaac8>>
CPU times: user 6.43 ms, sys: 3.06 ms, total: 9.5 ms
Wall time: 4.02 s


[3.0, 2.0]

In [34]:
%%time
# get using cache
cv([snap, new_snap])

2020-11-09 14:12:44,683 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7fa4e8fcaac8>>
CPU times: user 1.32 ms, sys: 823 µs, total: 2.14 ms
Wall time: 3.65 ms


[3.0, 2.0]

In [35]:
storage.save(cv)

2020-11-09 14:12:44,697 - openpathsampling.experimental.simstore.storage - DEBUG - Starting save
2020-11-09 14:12:44,705 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 1 UUIDs
2020-11-09 14:12:44,706 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 1 UUIDs
2020-11-09 14:12:44,711 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 0 UUIDs
2020-11-09 14:12:44,733 - openpathsampling.experimental.simstore.storage - DEBUG - Listing all objects to save
2020-11-09 14:12:44,739 - openpathsampling.experimental.simstore.storage - DEBUG - Checking if objects already exist in database
2020-11-09 14:12:44,741 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 2 UUIDs
2020-11-09 14:12:44,742 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 2 UUIDs
2020-11-09 14:12:44,749 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 0 UUIDs
2020-11-09 14:12:44,751 - openp

In [36]:
%%time
# get using storage
cv.local_cache.clear()
cv([snap, new_snap])

2020-11-09 14:12:44,868 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7fa4e8fcaac8>>
2020-11-09 14:12:44,871 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_storage of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7fa4e8fcaac8>>
2020-11-09 14:12:44,875 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 2 UUIDs
CPU times: user 5.77 ms, sys: 2.6 ms, total: 8.37 ms
Wall time: 9.75 ms


[3.0, 2.0]

## Save multiple times

The second time we save things, the `find_uuid` skips over the already-saved function. This means that we need another mechanism for storing mutable information (such as the `StorableFunctionResults`) to disk.

This is part of what the `StorableFunctionHandler` manages. Since the handler contains the function, it can store things to disk with an update.

In [37]:
backend = SQLStorageBackend("test.sql", mode='w')
storage = Storage.from_backend(
    backend=backend,
    schema=ops_schema,
    class_info=ops_class_info
)
cv = StorableFunction(get_x)
vol = paths.CVDefinedVolume(cv, lambda_min=0.0, lambda_max=1.0)
cv(snap)
storage.save(vol)

2020-11-09 14:12:44,913 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table uuid
2020-11-09 14:12:44,916 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table tables
2020-11-09 14:12:44,941 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table samples
2020-11-09 14:12:44,962 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table sample_sets
2020-11-09 14:12:44,978 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table trajectories
2020-11-09 14:12:44,989 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table move_changes
2020-11-09 14:12:45,003 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table steps
2020-11-09 14:12:45,012 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table details
2020-11-09 14:12:45,024 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table s

In [38]:
new_snap = toys.Snapshot(coordinates=np.array([[2.0, 0.0]]))
cv(new_snap)

2020-11-09 14:12:47,261 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7fa4e9125160>>
2020-11-09 14:12:47,262 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_storage of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7fa4e9125160>>
2020-11-09 14:12:47,268 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 0 UUIDs
2020-11-09 14:12:47,269 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._eval of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7fa4e9125160>>


2.0

In [39]:
storage.backend.load_storable_function_table(get_uuid(cv))

{'80100521955429081778427985284781047824': 3.0}

In [40]:
storage.save(vol)

2020-11-09 14:12:49,326 - openpathsampling.experimental.simstore.storage - DEBUG - Starting save
2020-11-09 14:12:49,328 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 1 UUIDs
2020-11-09 14:12:49,330 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 1 UUIDs
2020-11-09 14:12:49,350 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 1 UUIDs


In [41]:
storage.backend.load_storable_function_table(get_uuid(cv))

{'80100521955429081778427985284781047824': 3.0}

In [42]:
storage.save_function_results()

2020-11-09 14:12:49,427 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 0 UUIDs


In [43]:
storage.backend.load_storable_function_table(get_uuid(cv))

{'80100521955429081778427985284781047824': 3.0,
 '80100521955429081778427985284781047858': 2.0}

In [44]:
list(storage._sf_handler.canonical_functions.values())[0].local_cache.result_dict

{'80100521955429081778427985284781047824': 3.0,
 '80100521955429081778427985284781047858': 2.0}

## Mock of multiple computers creating and returning results

In [45]:
cv = StorableFunction(get_x)
cv(snap)

2020-11-09 14:12:49,467 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7fa4e91934a8>>
2020-11-09 14:12:49,468 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_storage of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7fa4e91934a8>>
2020-11-09 14:12:49,469 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._eval of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7fa4e91934a8>>


3.0

### Computer 1

We mock the serialization and transfer over the network by doing a `to_dict`/`from_dict` cycle, which will create a new instance in memory with the same UUID.

In [46]:
cv_1 = StorableFunction.from_dict(cv.to_dict())
set_uuid(cv_1, get_uuid(cv))

In [47]:
print("\n".join([repr(cv), repr(cv_1)]))

In [48]:
cv_1.local_cache.result_dict

{}

In [49]:
snap_1 = toys.Snapshot(coordinates=np.array([[4.0, 0.0]]))

In [50]:
%%time
cv_1(snap_1)

2020-11-09 14:12:51,570 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7fa4e917c748>>
2020-11-09 14:12:51,573 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_storage of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7fa4e917c748>>
2020-11-09 14:12:51,575 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._eval of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7fa4e917c748>>
CPU times: user 4.5 ms, sys: 2.13 ms, total: 6.62 ms
Wall time: 2.01 s


4.0

In [51]:
cv_1.local_cache.result_dict

{'80100521955429081778427985284781047868': 4.0}

### Computer 2

In [52]:
cv_2 = StorableFunction.from_dict(cv.to_dict())
set_uuid(cv_2, get_uuid(cv))

In [53]:
print("\n".join([repr(cv), repr(cv_1), repr(cv_2)]))

In [54]:
snap_2 = toys.Snapshot(coordinates=np.array([[5.0, 0.0]]))

In [55]:
cv_2(snap_2)

2020-11-09 14:12:53,693 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7fa4e90ef048>>
2020-11-09 14:12:53,703 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_storage of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7fa4e90ef048>>
2020-11-09 14:12:53,705 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._eval of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7fa4e90ef048>>


5.0

In [56]:
cv_2.local_cache.result_dict

{'80100521955429081778427985284781047874': 5.0}

### Gather operation

Note that for the real thing, we'll need to ensure that the serializers/deserializers (be they from `dask` or custom) correctly recreate the objects.

Attention: The save command must be issued separately for each copy of the CV. This is because the CVs have the same UUID, and we use sets/dicts to combine repeats of the same UUID.

In [57]:
backend = SQLStorageBackend("test.sql", mode='w')
storage = Storage.from_backend(
    backend=backend,
    schema=ops_schema,
    class_info=ops_class_info
)

2020-11-09 14:12:55,745 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table uuid
2020-11-09 14:12:55,746 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table tables
2020-11-09 14:12:55,758 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table samples
2020-11-09 14:12:55,771 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table sample_sets
2020-11-09 14:12:55,779 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table trajectories
2020-11-09 14:12:55,786 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table move_changes
2020-11-09 14:12:55,795 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table steps
2020-11-09 14:12:55,803 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table details
2020-11-09 14:12:55,822 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table s

In [58]:
storage.save([cv])

2020-11-09 14:12:56,024 - openpathsampling.experimental.simstore.storage - DEBUG - Starting save
2020-11-09 14:12:56,025 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 1 UUIDs
2020-11-09 14:12:56,026 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 1 UUIDs
2020-11-09 14:12:56,030 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 0 UUIDs
2020-11-09 14:12:56,031 - openpathsampling.experimental.simstore.storage - DEBUG - Listing all objects to save
2020-11-09 14:12:56,032 - openpathsampling.experimental.simstore.storage - DEBUG - Checking if objects already exist in database
2020-11-09 14:12:56,032 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 2 UUIDs
2020-11-09 14:12:56,034 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 2 UUIDs
2020-11-09 14:12:56,054 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 0 UUIDs
2020-11-09 14:12:56,056 - openp

In [59]:
storage.save([cv_1])

2020-11-09 14:12:56,133 - openpathsampling.experimental.simstore.storage - DEBUG - Starting save
2020-11-09 14:12:56,134 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 1 UUIDs
2020-11-09 14:12:56,135 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 1 UUIDs
2020-11-09 14:12:56,141 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 1 UUIDs
2020-11-09 14:12:56,143 - openpathsampling.experimental.simstore.storage - DEBUG - Listing all objects to save
2020-11-09 14:12:56,144 - openpathsampling.experimental.simstore.storage - DEBUG - Checking if objects already exist in database
2020-11-09 14:12:56,147 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 1 UUIDs
2020-11-09 14:12:56,150 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 1 UUIDs
2020-11-09 14:12:56,160 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 0 UUIDs
2020-11-09 14:12:56,161 - openp

In [60]:
storage.save([cv_2])

2020-11-09 14:12:56,193 - openpathsampling.experimental.simstore.storage - DEBUG - Starting save
2020-11-09 14:12:56,197 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 1 UUIDs
2020-11-09 14:12:56,199 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 1 UUIDs
2020-11-09 14:12:56,204 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 1 UUIDs
2020-11-09 14:12:56,206 - openpathsampling.experimental.simstore.storage - DEBUG - Listing all objects to save
2020-11-09 14:12:56,208 - openpathsampling.experimental.simstore.storage - DEBUG - Checking if objects already exist in database
2020-11-09 14:12:56,210 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 1 UUIDs
2020-11-09 14:12:56,211 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 1 UUIDs
2020-11-09 14:12:56,215 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 0 UUIDs
2020-11-09 14:12:56,217 - openp

In [61]:
cv_2.local_cache.parent

In [62]:
cv.local_cache.result_dict

{'80100521955429081778427985284781047824': 3.0,
 '80100521955429081778427985284781047868': 4.0,
 '80100521955429081778427985284781047874': 5.0}

In [63]:
storage._sf_handler.canonical_functions

{'80100521955429081778427985284781047860': <openpathsampling.experimental.simstore.storable_functions.StorableFunction at 0x7fa4e91934a8>}

In [64]:
print("\n".join([str(getattr(obj, '__uuid__')) for obj in [cv, cv_reloaded, cv_1, cv_2]]))

80100521955429081778427985284781047860
80100521955429081778427985284781047820
80100521955429081778427985284781047860
80100521955429081778427985284781047860


In [65]:
results = storage._sf_handler.canonical_functions[get_uuid(cv)].local_cache.result_dict

In [66]:
assert len(results) == 3

In [67]:
results

{'80100521955429081778427985284781047824': 3.0,
 '80100521955429081778427985284781047868': 4.0,
 '80100521955429081778427985284781047874': 5.0}

In [68]:
# should be Dict[str, List[StorableFunction]]
# dict length 1; the single value in dict is list length 3
storage._sf_handler.all_functions

defaultdict(list,
            {'80100521955429081778427985284781047860': [<openpathsampling.experimental.simstore.storable_functions.StorableFunction at 0x7fa4e91934a8>,
              <openpathsampling.experimental.simstore.storable_functions.StorableFunction at 0x7fa4e90ef048>]})

In [69]:
assert len(storage._sf_handler.all_functions) == 1
assert len(list(storage._sf_handler.all_functions.values())[0]) == 3

In [70]:
storage._sf_handler.clear_non_canonical()

In [71]:
storage._sf_handler.all_functions

defaultdict(list,
            {'80100521955429081778427985284781047860': [<openpathsampling.experimental.simstore.storable_functions.StorableFunction at 0x7fa4e91934a8>]})

In [72]:
assert len(storage._sf_handler.all_functions) == 1
assert len(list(storage._sf_handler.all_functions.values())[0]) == 1